In [7]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event,text
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *
from Connection import *


In [10]:
def ConnectSharePoint(url_hub):
    header_Hub = f"share_point_{url_hub.split('/')[2].replace('-','')}"
    config_Hub = read_config_json(config_path, header_Hub)
    Hub = SharePoint(config_Hub)
    #Hub.check_connect()
    return Hub


In [28]:

def ConnectAzureSQLServer():
    f = open('database_information.json', "r")
    qq = json.loads(f.read())
    f.close()
    ini_cnt_str = 'Driver={driver_str};Server=tcp:{servername},1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
    quoted = quote_plus(ini_cnt_str)
    cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
    engine = create_engine(cnt_str)
    return engine

def get_column_data_types_from_db(table_schema, table_name):
    engine = ConnectAzureSQLServer()
    # Retrieve column information from the database
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{table_schema}' and TABLE_NAME = '{table_name}'"))
        column_info = result.fetchall()

    # Extract column names and data types
    column_data_types = {}
    for col in column_info:
        column_name = col[0]
        data_type = col[1]
        column_data_types[column_name] = data_type

    return column_data_types

def validate_excel_files(table_schema, table_name, file_paths):
    column_data_types = get_column_data_types_from_db(table_schema,table_name)
    del column_data_types['Surrogate_ID']
    for file_path in file_paths:
        try:
            # Read the Excel file
            df = pd.read_excel(file_path)

            # Validate data types
            for column in column_data_types:
                print(column)
                expected_type = column_data_types[column]
                mismatched_rows = df[column].loc[df[column].apply(lambda x: type(x) != expected_type)]
                if not mismatched_rows.empty:
                    for row_index in mismatched_rows.index:
                        print(f"Data type mismatch in column '{column}' at row {row_index} of file '{file_path}'")

            # Additional validation checks can be added based on your specific requirements

        except Exception as e:
            print(f"Error occurred while validating file '{file_path}': {str(e)}")
table_schema = input('Enter the table schema: ')
table_name = input('Enter the table name: ')
# Provide a list of file paths to validate
file_paths = ['IP_Bac Giang_20230331.xlsx']
# Call the validation function
validate_excel_files(table_schema,table_name, file_paths)

Date_Key
Data type mismatch in column 'Date_Key' at row 0 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 1 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 2 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 3 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 4 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 5 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 6 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 7 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 8 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 9 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at row 10 of file 'IP_Bac Giang_20230331.xlsx'
Data type mismatch in column 'Date_Key' at r